In [ ]:
import streamlit as st
import mysql.connector
import pandas as pd

# Database Configuration - Replace with your actual details
DB_CONFIG = {
    'host': 'localhost',        # Replace with your MySQL host if different
    'user': 'your_mysql_user',   # Replace with your MySQL username
    'password': 'your_mysql_password', # Replace with your MySQL password
    'database': 'your_database_name'  # Replace with your database name
}


# Function to connect to the MySQL database
def connect_to_db():
    try:
        cnx = mysql.connector.connect(**DB_CONFIG)
        print("Successfully connected to the database.")
        return cnx
    except mysql.connector.Error as err:
        print(f"Error connecting to the database: {err}")
        return None

# Function to load data from the Customers table
def load_customers_data():
    cnx = connect_to_db()
    if cnx:
        try:
            query = "SELECT * FROM Customers"
            df = pd.read_sql(query, cnx)
            return df
        except Exception as e:
            st.error(f"Error loading customer data: {e}")
            return None
        finally:
            cnx.close()
    return None

# Function to load data from the Restaurants table
def load_restaurants_data():
    cnx = connect_to_db()
    if cnx:
        try:
            query = "SELECT * FROM Restaurants"
            df = pd.read_sql(query, cnx)
            return df
        except Exception as e:
            st.error(f"Error loading restaurant data: {e}")
            return None
        finally:
            cnx.close()
    return None

# Function to load data from the Orders table
def load_orders_data():
    cnx = connect_to_db()
    if cnx:
        try:
            query = "SELECT * FROM Orders"
            df = pd.read_sql(query, cnx)
            return df
        except Exception as e:
            st.error(f"Error loading order data: {e}")
            return None
        finally:
            cnx.close()
    return None

# Function to add a new customer
def add_customer(name, email, phone, location):
    cnx = connect_to_db()
    if cnx:
        cursor = cnx.cursor()
        try:
            query = """
            INSERT INTO Customers (name, email, phone, location, signup_date, is_premium, preferred_cuisine, total_orders, average_rating)
            VALUES (%s, %s, %s, %s, CURDATE(), 0, NULL, 0, 0.0)  # Using CURDATE() for current date
            """
            values = (name, email, phone, location)
            cursor.execute(query, values)
            cnx.commit()
            st.success('Customer added successfully!')
        except mysql.connector.Error as err:
            st.error(f"Error adding customer: {err}")
        finally:
            cursor.close()
            cnx.close()

# Main Streamlit app
def main():
    st.title("Delivery Data Dashboard")

    # Customer Data Section
    st.header("Customers")
    customers_df = load_customers_data()
    if customers_df is not None:
        st.dataframe(customers_df)  # Use dataframe instead of write
        # Add download button
        @st.cache_data  # Use cache_data instead of cache
        def convert_df(df):
            return df.to_csv(index=False).encode('utf-8')

        csv = convert_df(customers_df)

        st.download_button(
            label="Download Customer Data as CSV",
            data=csv,
            file_name='customers.csv',
            mime='text/csv',
        )
    else:
        st.warning("Could not load customer data.")

    # Restaurant Data Section
    st.header("Restaurants")
    restaurants_df = load_restaurants_data()
    if restaurants_df is not None:
        st.dataframe(restaurants_df)  # Use dataframe instead of write

        @st.cache_data
        def convert_df(df):
            return df.to_csv(index=False).encode('utf-8')

        csv = convert_df(restaurants_df)

        st.download_button(
            label="Download Restaurants Data as CSV",
            data=csv,
            file_name='restaurants.csv',
            mime='text/csv',
        )

    else:
        st.warning("Could not load restaurant data.")

    # Order Data Section
    st.header("Orders")
    orders_df = load_orders_data()
    if orders_df is not None:
        st.dataframe(orders_df)  # Use dataframe instead of write

        @st.cache_data
        def convert_df(df):
            return df.to_csv(index=False).encode('utf-8')

        csv = convert_df(orders_df)

        st.download_button(
            label="Download Orders Data as CSV",
            data=csv,
            file_name='orders.csv',
            mime='text/csv',
        )

    else:
        st.warning("Could not load order data.")


    # Add Customer Form
    st.header("Add New Customer")
    with st.form("add_customer_form"): # use forms for more structured input
        customer_name = st.text_input("Name")
        customer_email = st.text_input("Email")
        customer_phone = st.text_input("Phone")
        customer_location = st.text_input("Location")
        submitted = st.form_submit_button("Add Customer")
        if submitted:
            add_customer(customer_name, customer_email, customer_phone, customer_location)
            # Refresh the data after adding a customer (optional, but good UX)
            st.experimental_rerun() # Streamlit rerun


if __name__ == "__main__":
    main()